In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Explaining image classification with Vertex Explainable AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/xai_image_classification_feature_attributions.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fexplainable_ai%2fxai_image_classification_feature_attributions.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/explainable_ai/xai_image_classification_feature_attributions.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/xai_image_classification_feature_attributions.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview

Vertex Explainable AI offers feature-based and example-based explanations to provide better understanding of model decision making. For feature-based explanations, Vertex Explainable AI integrates feature attributions into Vertex AI. Feature attributions indicate how much each feature in your model contributed to the predictions for each given instance. For an image classification model, when you request explanations, you get the predicted class along with an overlay for the image, showing which areas in the image contributed most strongly to the resulting prediction.

To use Vertex Explainable AI on a pre-trained or custom-trained model, you must configure certain options when you create the model resource that you plan to request explanations from, when you deploy the model, or when you submit a batch explanation job. This tutorial demonstrates how to configure these options, and get and visualize explanations for online and batch predictions.

Learn more about [Vertex Explainable AI](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview) and [Vertex AI Prediction](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions).

### Objective

In this tutorial, you learn how to configure feature-based explanations on a pre-trained image classification model and make online and batch predictions with explanations.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex Explainable AI
- Vertex AI Prediction

The steps performed include:

- Download pretrained model from TensorFlow Hub
- Upload model for deployment
- Deploy model for online prediction
- Make online prediction with explanations
- Make batch predictions with explanations


### Dataset

In this example, you use the TensorFlow [flowers](http://download.tensorflow.org/example_images/flower_photos.tgz) dataset. The dataset contains about 3,700 photos of flowers in five sub-directories, one per class.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [ ]:
# Install the packages
! pip3 install --upgrade -q google-cloud-aiplatform \
                            tensorflow==2.15.1 \
                            tensorflow-hub \
                            matplotlib

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

### Import libraries

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from google.cloud import aiplatform

### Initialize Vertex AI SDK for Python

To get started using Vertex AI, you must [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Download pre-trained model from TensorFlow Hub

Feature attribution is supported for all types of models (both AutoML and custom-trained), frameworks (TensorFlow, scikit, XGBoost), and modalities (images, text, tabular, video).

For demonstration purposes, this tutorial uses an image model [Inception_v3](https://tfhub.dev/google/imagenet/inception_v3/classification/5) from the TensorFlow Hub. This model was pre-trained on the ImageNet benchmark dataset.
First, you download the model from the TensorFlow Hub, wrap it as a Keras layer with `hub.KerasLayer` and save the model artifacts to your Cloud Storage bucket.

In [ ]:
classifier_model = "https://tfhub.dev/google/imagenet/inception_v3/classification/5"

classifier = tf.keras.Sequential([hub.KerasLayer(classifier_model)])

classifier.build([None, 224, 224, 3])

MODEL_DIR = f"{BUCKET_URI}/model"
classifier.save(MODEL_DIR)

## Upload model for deployment

Next, you upload the model to Vertex AI Model Registry, which will create a Vertex AI Model resource for your model. Prior to uploading, you need to define a serving function to convert data to the format your model expects.

### Define a serving function for image data

You define a serving function to convert image data to the format your model expects. When you send encoded data to Vertex AI, your serving function ensures that the data is decoded on the model server before it is passed as input to your model.

To enable Vertex Explainable AI in your custom models, you need to set two additional signatures from the serving function:

- `xai_preprocess`: The preprocessing function in the serving function.
- `xai_model`: The concrete function for calling the model.

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(224, 224))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(classifier.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    classifier,
    MODEL_DIR,
    signatures={
        "serving_default": serving_fn,
        "xai_preprocess": preprocess_fn,  # Required for XAI
        "xai_model": m_call,  # Required for XAI
    },
)

### Get the serving function signature

You get the signatures of your model's input and output layers by reloading the model into memory, and querying it for the signatures corresponding to each layer. The input layer name of the serving function will be used later when you make a prediction request.

In [ ]:
loaded = tf.saved_model.load(MODEL_DIR)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

## Configure explanation settings

To use Vertex Explainable AI with a custom-trained model, you must configure explanation settings when uploading the model. These settings include:

- `parameters`: The feature attribution method. Available methods include `shapley`, `ig`, `xrai`.
- `metadata`: The model's input and output for explanation. **This field is optional for TensorFlow 2 models. If omitted, Vertex AI automatically infers the inputs and outputs from the model**. You don't need to configure this field in this tutorial.

Learn more about [configuring feature-based explanations](https://cloud.google.com/vertex-ai/docs/explainable-ai/configuring-explanations-feature-based).

In [ ]:
XAI = "ig"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": 10}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": 50}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": 50}}

parameters = aiplatform.explain.ExplanationParameters(PARAMETERS)

## Upload the model to a Vertex AI Model resource

Next, upload your model to a Vertex AI Model resource with the explanation configuration. Vertex AI provides [Docker container images](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers) that you run as pre-built containers for serving predictions and explanations from trained model artifacts.

In [ ]:
MODEL_DISPLAY_NAME = "inception-v3-model-unique"
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-11:latest"

model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    explanation_parameters=parameters,
)

## Deploy model for online prediction

Next, deploy your model for online prediction. You set the variable `DEPLOY_COMPUTE` to configure the machine type for the [compute resources](https://cloud.google.com/vertex-ai/docs/predictions/configure-compute) you will use for prediction.

In [ ]:
DEPLOY_DISPLAY_NAME = "inception-v3-deploy-unique"
DEPLOY_COMPUTE = "n1-standard-4"

endpoint = model.deploy(
    deployed_model_display_name=DEPLOY_DISPLAY_NAME,
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=None,
    accelerator_count=0,
)

## Make online prediction with explanations


### Download an image dataset and labels
In this example, you use the TensorFlow flowers dataset for the input data for predictions. The dataset contains about 3,700 photos of flowers in five sub-directories, one per class. You also fetch the ImageNet dataset labels to decode the predictions.


In [ ]:
import pathlib

data_dir = tf.keras.utils.get_file(
    "flower_photos",
    origin="https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz",
    untar=True,
)

data_dir = pathlib.Path(data_dir)
images_files = list(data_dir.glob("daisy/*"))

labels_path = tf.keras.utils.get_file(
    "ImageNetLabels.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt",
)

imagenet_labels = np.array(open(labels_path).read().splitlines())

### Prepare image processing functions

You define some reusable functions for image processing and visualization.

In [ ]:
import base64
import io
import json

import matplotlib.image as mpimg
import matplotlib.pyplot as plt


def encode_image_in_b64(image_file):
    bytes = tf.io.read_file(image_file)
    b64str = base64.b64encode(bytes.numpy()).decode("utf-8")
    return b64str


def decode_b64_image(b64_image_str):
    image = base64.b64decode(b64_image_str)
    image = io.BytesIO(image)
    image = mpimg.imread(image, format="JPG")
    return image


def decode_numpy_image(numpy_inputs):
    numpy_inputs_json = json.loads(str(numpy_inputs))
    image = np.array(numpy_inputs_json)
    return image


def show_explanation(encoded_image, prediction, feature_attributions):
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(16, 4))

    label_index = np.argmax(prediction)
    class_name = imagenet_labels[label_index]
    confidence_score = prediction[label_index]
    axs[0].set_title(
        "Prediction:[" + class_name + "] (" + str(round(confidence_score, 1)) + "%)"
    )
    original_image = decode_b64_image(encoded_image)
    axs[0].imshow(original_image, interpolation="nearest", aspect="auto")
    axs[0].axis("off")

    numpy_inputs = feature_attributions["numpy_inputs"]
    attribution_image = decode_numpy_image(numpy_inputs)
    axs[1].set_title("Feature attributions")
    axs[1].imshow(attribution_image, interpolation="nearest", aspect="auto")
    axs[1].axis("off")

    processed_image = attribution_image.max(axis=2)
    axs[2].imshow(processed_image, cmap="coolwarm", aspect="auto")
    axs[2].set_title("Feature attribution heatmap")
    axs[2].axis("off")

### Get online explanations

You send an `explain` request with encoded input image data to the `endpoint` and get predictions with explanations.

In [ ]:
TEST_IMAGE_SIZE = 2

test_image_list = []
for i in range(TEST_IMAGE_SIZE):
    test_image_list.append(str(images_files[i]))

instances_list = []
for test_image in test_image_list:
    b64str = encode_image_in_b64(test_image)
    instances_list.append({serving_input: {"b64": b64str}})

response = endpoint.explain(instances_list)

### Visualize online explanations

As you request explanations on an image classification model, you get the predicted class along with an image overlay showing which pixels (integrated gradients) or regions (integrated gradients or XRAI) contributed to the prediction.

In [ ]:
for i, test_image in enumerate(test_image_list):
    encoded_image = encode_image_in_b64(test_image)
    prediction = response.predictions[i]
    explanation = response.explanations[i]
    feature_attributions = dict(explanation.attributions[0].feature_attributions)

    show_explanation(encoded_image, prediction, feature_attributions)

## Make batch predictions with explanations

### Create the batch input file

You create a batch input file in JSONL format and store the input file in your Cloud Storage bucket.

In [ ]:
gcs_input_uri = f"{BUCKET_URI}/test_images.json"

with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    for test_image in test_image_list:
        b64str = encode_image_in_b64(test_image)
        data = {serving_input: {"b64": b64str}}
        f.write(json.dumps(data) + "\n")

### Submit a batch prediction job

You make a batch prediction by submitting a batch prediction job with the `generate_explanation` parameter set to `True`.

In [ ]:
JOB_DISPLAY_NAME = "inception-v3-job-unique"

batch_predict_job = model.batch_predict(
    job_display_name=JOB_DISPLAY_NAME,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="jsonl",
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
    generate_explanation=True,
)

### Get batch explanations

Next, you get the explanations from the completed batch prediction job. The results are written to the Cloud Storage output bucket you specified in the batch prediction request. You call the method `iter_outputs()` to get a list of each Cloud Storage file generated with the results.

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

explanation_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("explanation.results"):
        explanation_results.append(blob.name)

### Visualize explanations

You take one explanation result as an example and visualize the explanations. For an image classification model, you get the predicted class along with an image overlay showing which pixels (integrated gradients) or regions (integrated gradients or XRAI) contributed to the prediction.

In [ ]:
explanation_result = explanation_results[0]

gfile_name = f"{BUCKET_URI}/{explanation_result}"
with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
    result = json.loads(gfile.read())

encoded_image = result["instance"]["bytes_inputs"]["b64"]
prediction = result["prediction"]
attributions = result["explanation"]["attributions"][0]
feature_attributions = attributions["featureAttributions"]

show_explanation(encoded_image, prediction, feature_attributions)

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
# Deploy the model from endpoint
endpoint.undeploy_all()
# Delete the endpoint
endpoint.delete()
# Delete the model
model.delete()
# Delete the batch prediction job
batch_predict_job.delete()

# Delete Cloud Storage bucket
delete_bucket = False  # Set True to delete your bucket
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI